## Created by yunsuxiaozi 2024/3/12
### 这里将实现SM3密码杂凑算法.关于算法的详细介绍可以看PDF,我这里也是看PDF慢慢写出来的,并且用后面的测试样例来测试,能通过就基本没有问题了.SM3算法和SHA256算法大体的步骤非常类似,在实现的细节上不同.

### 无法连接到github可以尝试这个命令:  cmd:  ipconfig /flushdns

### 本地的代码上传到github后删除掉,需要 git reset +git checkout .这两个命令将仓库的文件下载到本地后再git add新的文件,commit(修改信息) push(上传).

#### 这是循环左移的函数

In [1]:
"""
^(按位与)的python符号是&.
√(按位或)的python符号是|
⊕(异或)的python符号是^.
"""
inf=0xFFFFFFFF#32位无符号整数的最大值(2^32-1)
#循环左移  x=[a1,a2,……,a32] 
def ROTL(x, n):
    x=x&inf#如果输入的数据太大,会限制到2^32-1的范围内.
    n=n%32#如果n>32,则限制到32以内的范围.
    #x=[a1,a2,……,a32]  则x>>(32-n) =[0,0,……,0,a1,a2,……,an] 
    #x<<(n) =[an+1,……a32,0,0,……,0]
    x = ((x << n) | (x >> (32 - n)))&inf
    return x

#### 这是根据PDF实现的,我也不知道算法是怎么设计出来的,同时这个算法和sha256算法非常类似,如果有需要可以看看sha256算法中我对算法的理解.这个算法我是昨天晚上花了2个小时写出来的,也没有像sha256算法思考的那么久.

In [2]:
def FF(j,X,Y,Z):
    X,Y,Z=(X&inf),(Y&inf),(Z&inf)
    if j<16:
        return (X^Y^Z &inf)
    else:
        return ((X  & Y)|( X & Z)|(Y&Z)&inf)

def GG(j,X,Y,Z):
    X,Y,Z=(X&inf),(Y&inf),(Z&inf)
    if j<16:
        return (X^Y^Z&inf)
    else:
        return ((X  & Y)|(~X & Z)&inf)

def P0(X):
    X=X&inf
    return ((X^ROTL(X,9)&inf)^ROTL(X,17))&inf
def P1(X):
    X=X&inf
    return ((X^ROTL(X,15)&inf)^ROTL(X,23))&inf

def SM3(origin_str='helloworld'):
    V=[0x7380166f,0x4914b2b9,0x172442d7,0xda8a0600,0xa96f30bc,0x163138aa,0xe38dee4d,0xb0fb0e4e]
    T=[0x79cc4519 if j<16 else 0x7a879d8a for j in range(0,64)]
    #utf-8 能够表示世界上几乎所有的字符系统,无论原始字符串是什么,都能够得到一个统一的二进制表示.
    binaries = origin_str.encode('utf8')
    """
    \x80(16进制)=128(10进制)=1000 0000(2进制,8位)
    b'\x00'=00000000(2进制 8位)
    len(binaries)是字节串的长度,即有多少字节,*8是每个字节由8位组成,转换成位长度
    to_bytes(8)转换成8字节或者说64位的字节串.byteorder='big'是大端字节序(就是从头到尾存储).

    位长度: len(binaries)*8+8位+8位*(64-len(binaries)-1-8)+64位=512.

    """
    churks=1#M是512的多少倍

    while 64*churks-len(binaries)-9<0:
        churks+=1
    #M是512的整数倍
    M = binaries + b'\x80' + b'\x00'*(64*churks-len(binaries)-1-8) + (len(binaries)<<3).to_bytes(8, byteorder='big')

    for churk in range(churks):
        M_copy=M[churk*64:(churk+1)*64]
        W=[0]*68
        W1=[0]*64
        #W[0]~W[15] 这里已经把 这个churk的二进制转成整数
        for j in range(0, 16):
            W[j] = M_copy[j*4:(j+1)*4]#4*8=32位长度.
            #W[t]转成16进制再转成10进制.
            W[j] = int(W[j].hex(), 16)#2^32的整数.
        for j in range(16,68):
            W[j]=(P1( W[j-16] ^ W[j-9] ^ROTL(W[j-3],15)) )  ^ROTL(W[j-13],7)^(W[j-6]&inf)
              
        for j in range(0,64):
            W1[j]=(W[j]^W[j+4])&inf
    
        a,b,c,d,e,f,g,h=V
        for j in range(64):#
            SS1=(   ROTL( ( ROTL(a,12)+e+ROTL(T[j],j))&inf  ,7)    ) &inf
            SS2=(SS1^ROTL(a,12))&inf
            TT1=(FF(j,a,b,c)+d+SS2+W1[j])&inf
            TT2=(GG(j,e,f,g)+h+SS1+W[j])&inf
            d=c
            c=ROTL(b,9)
            b=a
            a=TT1
            h=g
            g=ROTL(f,19)
            f=e
            e=P0(TT2)
            #print(f"j:{j},a:{hex(a)},b:{hex(b)},c:{hex(c)},d:{hex(d)},e:{hex(e)},f:{hex(f)},g:{hex(g)},h:{hex(h)}")
        V[0]=a^V[0]&inf
        V[1]=b^V[1]&inf
        V[2]=c^V[2]&inf
        V[3]=d^V[3]&inf
        V[4]=e^V[4]&inf
        V[5]=f^V[5]&inf
        V[6]=g^V[6]&inf
        V[7]=h^V[7]&inf
            
    SM3 = ''
    for idx  in  range(len(V)):
        #将整数转换成4个字节(4*8=32位)的二进制数据,然后再转成十六进制数据(hex)
        SM3 = SM3+ V[idx].to_bytes(4, byteorder='big').hex()
    return SM3

#### SM3是我国实现的算法,接下来我们来看看手写的算法对比python标准库的准确率.

In [3]:
#!pip install是安装python库 -q:减少输出信息,gmssl是密码工具库,支持中国的国家商用密码（国密）算法.
!pip install -q gmssl

In [4]:
from gmssl import sm3, func

import random#用于生成随机数的库
seed=2024
random.seed(seed)
num_string=10000
char_num="abcdefghijklmnopqrstuvwxyz0123456789"#可供选择的字符是a~z加上数字
random_strings = []#存储随机生成的字符串
for _ in range(num_string):
    str_len = random.randint(10, 500)  #生成长度为10到500的字符
    random_string = ''.join(random.choices(char_num, k=str_len))
    random_strings.append(random_string)
    
true_cnt=0#手写算法正确的个数
for idx in range(num_string):
    origin_str=random_strings[idx]
    if SM3(origin_str) == sm3.sm3_hash(func.bytes_to_list(origin_str.encode('utf8'))):
        true_cnt+=1
print(f"accuracy:{true_cnt/num_string}")

accuracy:1.0
